In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import (BUSINESS_DAYS_IN_YEAR,
                                   standardDeviation)
from _utils.portfoliohandcrafiting import (
    create_fx_series_given_adjusted_prices_dict,
    calculate_variable_standard_deviation_for_risk_targeting_from_dict,
    calculate_position_series_given_variable_risk_for_dict,
    calculate_perc_returns_for_dict,
    calculate_perc_returns_for_dict_with_costs,
)
from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *
from _utils.strategies.trend_carry import *
from _utils.strategies.risk_adjustments import *


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [2]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data

In [3]:
adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict(['sp500','us10'])
multipliers = dict(sp500=5, us10=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, us10=1))

capital = 2000000
idm = 1.5
instrument_weights = dict(sp500=0.5, us10=0.5)
cost_per_contract_dict = dict(sp500=0.875, us10=5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices_dict,
current_prices=current_prices_dict,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [5]:
average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

rule_spec = [
        dict(function="carry", span=5),
        dict(function="carry", span=20),
        dict(function="carry", span=60),
        dict(function="carry", span=120),
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

position_contracts_carry_trend_riskadjust_dict = calculate_position_dict_with_forecast_and_vol_scalar_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        rule_spec=rule_spec,
        apply_vol_regime_to_carry=True,
        apply_vol_regime_to_emac=True,
    )


buffered_position_carry_trend_riskadjust_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_carry_trend_riskadjust_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_carry_trend_riskadjustdict = calculate_perc_returns_for_dict(
position_contracts_dict=buffered_position_carry_trend_riskadjust_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=adjusted_prices_dict,
)


fast_spans = [2, 4, 8, 16, 32, 64]
position_contracts_adjusted_trend_dict = (
    calculate_position_dict_with_multiple_trend_forecast_applied_and_adjustment(
        adjusted_prices_dict=adjusted_prices_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_spans=fast_spans,
    )
)

buffered_position_adjusted_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_adjusted_trend_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_adjusted_trend_dict = calculate_perc_returns_for_dict(
position_contracts_dict=buffered_position_adjusted_trend_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=adjusted_prices_dict,
)

fast_spans = [16, 32, 64]
position_contracts_multiple_trend_forecast_dict = (
       calculate_position_dict_with_multiple_trend_forecast_applied(
           adjusted_prices_dict=adjusted_prices_dict,
           average_position_contracts_dict=average_position_contracts_dict,
           std_dev_dict=std_dev_dict,
           fast_spans=fast_spans,
       )
   )

buffered_position_multiple_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_multiple_trend_forecast_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_multiple_trend_dict = calculate_perc_returns_for_dict(
position_contracts_dict=buffered_position_multiple_trend_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=adjusted_prices_dict,
)

port_return_carry_trend_riskadjust = aggregate_returns(perc_return_carry_trend_riskadjustdict)
port_return_carry_trend_riskadjust.sort_index(inplace=True)

port_return_adjusted_trend = aggregate_returns(perc_return_adjusted_trend_dict)
port_return_adjusted_trend.sort_index(inplace=True)

port_multiple_trend_forecast_return = aggregate_returns(perc_return_multiple_trend_dict)
port_multiple_trend_forecast_return.sort_index(inplace=True)

In [6]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=port_return_carry_trend_riskadjust.index,
        y=port_return_carry_trend_riskadjust.cumsum()*100,
        name="Trend & Carry with Risk Adjust",
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_return_adjusted_trend.index,
        y=port_return_adjusted_trend.cumsum()*100,
        name="Adjusted Trend",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_multiple_trend_forecast_return.index,
        y=port_multiple_trend_forecast_return.cumsum()*100,
        name="Multiple Carry Forecast",
        line=dict(color="red"),
    )
)



fig.update_layout(
    title="50% SP500 Fut + 50% US10 Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Carry and trend with Risk Adjust Frequência Anual")
print(calculate_stats(port_return_carry_trend_riskadjust, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Adjusted Trend Frequência Anual")
print(calculate_stats(port_return_adjusted_trend, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC com filtro de Multiplas Trends Frequência Anual")
print(calculate_stats(port_multiple_trend_forecast_return, freq=YEAR))





Estatísticas de Desempenho
Estatísticas Portfolio Carry and trend with Risk Adjust Frequência Anual
{'ann_mean': 28.188102429992636, 'ann_std': 3.842478172164344, 'sharpe': 7.335917386386918, 'skew': 0.7655844544752707, 'avg_drawdown': 0.055088264416042684, 'max_drawdown': 0.32538772804687577, 'quant_lower': 0.6360988652610841, 'quant_upper': 1.51594614757369}
Estatísticas Portfolio Adjusted Trend Frequência Anual
{'ann_mean': 28.291445635270644, 'ann_std': 4.116064092836835, 'sharpe': 6.873422035508655, 'skew': 0.6227173880920615, 'avg_drawdown': 0.11038035328359727, 'max_drawdown': 0.4651211709765626, 'quant_lower': 0.5434834627153602, 'quant_upper': 1.3203429045662316}
Estatísticas Portfolio EWMAC com filtro de Multiplas Trends Frequência Anual
{'ann_mean': 27.025355232199438, 'ann_std': 3.836794317224767, 'sharpe': 7.043733126603314, 'skew': 2.1180776915988853, 'avg_drawdown': 0.06655495294267874, 'max_drawdown': 0.3830852339257813, 'quant_lower': 0.4187294287370029, 'quant_upper':